In [1]:
import cv2
import numpy as np
import face_recognition

class HeadMovementTracker:
    frames_check = 2

    def __init__(self):
        self.frame_pos = 0
        self.prev_states = [False] * self.frames_check
        self.eye_distance = None
    def calculate_distance(self, point1, point2):
        return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)
    
    def detect_head_movement(self, frame):

        head_movement_detected = False  # Initialize the flag
        
        frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_landmarks = face_recognition.face_landmarks(frameRGB)

        if not face_landmarks:
            head_movement_detected = True
        else:
            landmarks = face_landmarks[0]  # Assuming only one face is detected
            left_eye = landmarks['left_eye']
            right_eye = landmarks['right_eye']

            if not left_eye or not right_eye:
                head_movement_detected = True
            else:
                current_eye_distance = self.calculate_distance(left_eye[0], right_eye[3])

                cv2.rectangle(frame, (left_eye[0][0], left_eye[0][1]),
                            (right_eye[3][0], right_eye[3][1]), (0, 0, 255), 2)

                if self.eye_distance is None:
                    self.eye_distance = current_eye_distance

                threshold = self.eye_distance * 0.1  # Set the threshold to 10% reduction

                distance_change = self.eye_distance - current_eye_distance

                if abs(distance_change) > threshold:
                    head_movement_detected = True

                self.eye_distance = current_eye_distance
        
        return head_movement_detected


    def eye_based_snooping_detect(self, human_frame):
        curr_state = self.perform_detection_fn_call_here()
        self.prev_states[self.frame_pos] = curr_state
        print(self.frame_pos)
        self.frame_pos = (self.frame_pos + 1) % self.frames_check

        if sum(self.prev_states) == self.frames_check:
            # Snooping for more than a certain period of time - reset the object and return
            self.frame_pos = 0
            self.prev_states = [False] * self.frames_check

            return True

        return False
